In [ ]:
pip install ultralytics

In [ ]:
import os
import pandas as pd
import shutil
from ultralytics import YOLO
import requests
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# convert function
def convert_and_organize(split_name):
    csv_path = f"/kaggle/input/license-plate-recognition/{split_name}/{split_name}/_annotations.csv"
    img_src = f"/kaggle/input/license-plate-recognition/{split_name}/{split_name}"
    img_dst = f"/kaggle/working/dataset/images/{split_name}"
    label_dst = f"/kaggle/working/dataset/labels/{split_name}"

    os.makedirs(img_dst, exist_ok=True)
    os.makedirs(label_dst, exist_ok=True)

    df = pd.read_csv(csv_path)

    for _, row in df.iterrows():
        filename = row['filename']
        width = row['width']
        height = row['height']
        xmin = row['xmin']
        ymin = row['ymin']
        xmax = row['xmax']
        ymax = row['ymax']

        # Convert to YOLO format
        x_center = ((xmin + xmax) / 2) / width
        y_center = ((ymin + ymax) / 2) / height
        box_width = (xmax - xmin) / width
        box_height = (ymax - ymin) / height

        yolo_line = f"0 {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}\n"

        # Save label file
        txt_file = os.path.splitext(filename)[0] + ".txt"
        with open(os.path.join(label_dst, txt_file), 'a') as f:
            f.write(yolo_line)

        # Copy image
        src_img_path = os.path.join(img_src, filename)
        dst_img_path = os.path.join(img_dst, filename)
        if os.path.exists(src_img_path):
            shutil.copy(src_img_path, dst_img_path)

# Run for all splits
for split in ['train', 'valid', 'test']:
    convert_and_organize(split)

In [ ]:
yaml_content = """
path: /kaggle/working/dataset
train: images/train
val: images/valid
test: images/test

names:
  0: license_plate
"""


with open("/kaggle/working/dataset.yaml", "w") as f:
    f.write(yaml_content)

In [ ]:
model = YOLO("yolov8n.pt") 

In [ ]:
model.train(
    data="/kaggle/working/dataset.yaml",
    epochs=30,
    imgsz=640,
    batch=32
)

In [ ]:
import shutil
shutil.copy("runs/detect/train/weights/best.pt", "my_custom_model.pt")

In [ ]:
model_path = "/kaggle/input/yolo/pytorch/default/1/my_custom_model.pt"
model = YOLO(model_path) 

from pathlib import Path

test_img_dir = Path("/kaggle/working/dataset/images/test")
test_label_dir = Path("/kaggle/working/dataset/labels/test")

test_images = list(test_img_dir.glob("*.jpg"))

In [ ]:
import numpy as np

def xywh2xyxy(x):
    # x: [x_center, y_center, width, height] normalized (0-1)
    y = np.copy(x)
    y[0] = x[0] - x[2] / 2  # x_min
    y[1] = x[1] - x[3] / 2  # y_min
    y[2] = x[0] + x[2] / 2  # x_max
    y[3] = x[1] + x[3] / 2  # y_max
    return y

def iou(boxA, boxB):
    # box: [x_min, y_min, x_max, y_max], values in absolute pixels or normalized (consistent)
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    if boxAArea + boxBArea - interArea == 0:
        return 0
    return interArea / (boxAArea + boxBArea - interArea)


In [ ]:
ious = []
all_scores = []
all_labels = []

iou_threshold = 0.5

for img_path in test_images:
    results = model(str(img_path))
    res = results[0]  # Get the Results object

    # Extract predictions: boxes, scores, classes
    pred_boxes = res.boxes.xyxy.cpu().numpy()  # absolute pixel coords [x1,y1,x2,y2]
    pred_scores = res.boxes.conf.cpu().numpy()
    pred_classes = res.boxes.cls.cpu().numpy()

    # Load ground truth boxes and convert YOLO format (normalized) to absolute pixel xyxy
    label_file = test_label_dir / (img_path.stem + ".txt")
    gt_boxes = []
    if label_file.exists():
        img = cv2.imread(str(img_path))
        img_h, img_w = img.shape[:2]
        with open(label_file) as f:
            for line in f:
                parts = line.strip().split()
                # YOLO format: class, x_center, y_center, width, height (normalized)
                _, x_c, y_c, w, h = map(float, parts)
                # Convert to absolute pixel coordinates
                x_c_abs = x_c * img_w
                y_c_abs = y_c * img_h
                w_abs = w * img_w
                h_abs = h * img_h

                x1 = x_c_abs - w_abs / 2
                y1 = y_c_abs - h_abs / 2
                x2 = x_c_abs + w_abs / 2
                y2 = y_c_abs + h_abs / 2

                gt_boxes.append([x1, y1, x2, y2])
    gt_boxes = np.array(gt_boxes)

    # For each predicted box, find IoU with GT boxes and assign TP/FP labels
    for i, pred_box in enumerate(pred_boxes):
        conf = pred_scores[i]
        all_scores.append(conf)

        if len(gt_boxes) == 0:
            all_labels.append(0)  # No GT boxes → false positive
            continue

        ious_for_pred = [iou(pred_box, gt_box) for gt_box in gt_boxes]
        max_iou = max(ious_for_pred) if ious_for_pred else 0
        ious.append(max_iou)

        if max_iou >= iou_threshold:
            all_labels.append(1)  # True positive
        else:
            all_labels.append(0)  # False positive


In [ ]:
print(type(results))
print(results)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, average_precision_score
import numpy as np

# Convert lists to numpy arrays
all_labels = np.array(all_labels)
all_scores = np.array(all_scores)

# Compute precision-recall curve
precision, recall, thresholds = precision_recall_curve(all_labels, all_scores)
ap = average_precision_score(all_labels, all_scores)

# Plot
plt.figure(figsize=(8,6))
plt.plot(recall, precision, label=f'AP = {ap:.3f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.grid(True)

plt.savefig('/kaggle/working/precision_recall_curve.png')  # Save figure
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
plt.hist(ious, bins=30, range=(0,1), color='blue', alpha=0.7)
plt.xlabel('IoU')
plt.ylabel('Count')
plt.title('IoU Distribution of Predicted Boxes vs Ground Truth')
plt.grid(True)

plt.savefig('/kaggle/working/iou_distribution.png')  # Save the figure
plt.show()


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random

# Randomly sample 4 images from the test set
random_imgs = random.sample(test_images, 4)

fig, axs = plt.subplots(1, 4, figsize=(20, 5))

for i, img_path in enumerate(random_imgs):
    img = cv2.imread(str(img_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    results = model(str(img_path))
    res = results[0]

    pred_boxes = res.boxes.xyxy.cpu().numpy()
    pred_scores = res.boxes.conf.cpu().numpy()

    label_file = test_label_dir / (img_path.stem + ".txt")
    gt_boxes = []
    img_h, img_w = img.shape[:2]

    if label_file.exists():
        with open(label_file) as f:
            for line in f:
                parts = line.strip().split()
                _, x_c, y_c, w, h = map(float, parts)
                x_c_abs = x_c * img_w
                y_c_abs = y_c * img_h
                w_abs = w * img_w
                h_abs = h * img_h
                x1 = int(x_c_abs - w_abs / 2)
                y1 = int(y_c_abs - h_abs / 2)
                x2 = int(x_c_abs + w_abs / 2)
                y2 = int(y_c_abs + h_abs / 2)
                gt_boxes.append([x1, y1, x2, y2])

    for (x1, y1, x2, y2) in gt_boxes:
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    for j, (x1, y1, x2, y2) in enumerate(pred_boxes.astype(int)):
        conf = pred_scores[j]
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(img, f'{conf:.2f}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    axs[i].imshow(img)
    axs[i].set_title(f'Test Image {i+1}')
    axs[i].axis('off')

plt.tight_layout()
plt.savefig('comparison.png')
plt.close()
